<a href="https://colab.research.google.com/github/Shubhpatel40/image_captioning/blob/main/image_captioning_using_custom_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adityajn105/flickr8k")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flickr8k


In [2]:
!apt install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libcudnn8
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 446 MB of archives.
After this operation, 1,140 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  libcudnn8 8.6.0.163-1+cuda11.8 [446 MB]
Fetched 446 MB in 8s (55.8 MB/s)
Selecting previously unselected package libcudnn8.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.6.0.163-1+cuda11.8_amd64.deb ...
Unpacking libcudnn8 (8.6.0.163-1+cuda11.8) ...
Setting up libcudnn8 (8.6.0.163-1+cuda11.8) ...


In [3]:
!pip uninstall -y tensorflow estimator keras

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0


In [4]:
!pip install -U tensorflow_text tensorflow tensorflow_datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 132.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow_text
    Found existing installation: tensorflow-text 2.18.1
    Uninstalling tensorflow-text-2.18.1:
      Successfully uninstalled tensorflow-text-2.18.1
ERROR: pip's dependency resolver does not currently take into account all the pa

In [5]:
!pip install einops

In [6]:
#@title
import concurrent.futures
import collections
import dataclasses
import hashlib
import itertools
import json
import math
import csv
import os
import pathlib
import random
import re
import string
import time
import urllib.request

import einops
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from PIL import Image
import requests
import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds
import sklearn

In [7]:
captions_file = path + "/captions.txt"

with open(captions_file, "r") as f:
    captions = f.read().splitlines()

In [8]:
full_dataset_dict = {}

for caption in captions[1:-1]:
    reader = csv.reader([caption])
    image, caption = tuple(next(reader))
    if image not in full_dataset_dict:
        full_dataset_dict[image] = []
    full_dataset_dict[image].append(caption)

### ***Load the model***

In [9]:
IMAGE_SHAPE = (224, 224, 3)
restnet = tf.keras.applications.ResNet50V2(
    input_shape=IMAGE_SHAPE,
    include_top=False)
restnet.trainable=False

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


***function to load the model and resize for the model***

In [10]:
def load_image(image_path):
    full_image_path = f"{path}/Images/{image_path}"
    img = tf.io.read_file(full_image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SHAPE[:-1])
    return img

In [11]:
print(tf.newaxis)

None


In [12]:
os.path.exists(f"{path}/Images/1000268201_693b08cb0e.jpg")

True

In [13]:
test_img_batch = load_image(list(full_dataset_dict.keys())[0])[tf.newaxis, :]

print(test_img_batch.shape)
print(restnet(test_img_batch).shape)

(1, 224, 224, 3)
(1, 7, 7, 2048)


## ***Preprocess Captions***

***Function for the remove punctuation, lower case, and add the special token before and after the captions.***

In [52]:
from typing import List
class preprocess_text:
   def __init__(self):
      pass

   def standardize(self, s:List):
      s = tf.constant(s, dtype=tf.string)
      s = tf.strings.lower(s)
      s = tf.strings.regex_replace(s, f'[{re.escape(string.punctuation)}]', '')
      byte_arr = s.numpy()

      # Decode each element safely
      decoded = [
          b.decode('utf-8') if isinstance(b, (bytes, bytearray)) else str(b)
          for b in byte_arr
      ]

      return [f"{t} [END]" for t in decoded]

   def make_vocabulary_dict(self, full_dataset_dict):
      vocabulary_size = 1
      vocab_dict = {'START': 0}
      for image in full_dataset_dict:
          for caption in full_dataset_dict[image]:
              for word in caption.split():
                  if word not in vocab_dict:
                      vocab_dict[word] = vocabulary_size
                      vocabulary_size += 1
                  else:
                      continue

      return vocab_dict

   def index_to_word(self, vocab_dict):
      index_to_word = {index: word for word, index in vocab_dict.items()}
      return index_to_word

   def word_to_index(self, s):
      return [vocab_dict[word] for word in s.split()]

In [47]:
class convert_output_to_tensor:
    def __init__(self, vocab_dict, vocabulary_size, max_len=50):
      self.vocab_dict = vocab_dict
      self.vocabulary_size = vocabulary_size
      self.max_len = max_len
      self.preprocess_text = preprocess_text()

    def preprocessing(self, caption_batch):
      all_captions_tokenIDs = []

      for caption in caption_batch:
        ## split the caption
        s_tokenID = []

        ## find the token ID for the each token from vocab_dict.
        for word in caption:
          if word in self.vocab_dict:
            s_tokenID.append(self.vocab_dict[word])

          else:
            s_tokenID.append(0)

        all_captions_tokenIDs.append(s_tokenID)

      tgt_ids = torch.tensor(all_captions_tokenIDs, dtype=torch.long)
      return tgt_ids

In [ ]:
# test_preprocess = preprocess_text(vocab_dict, vocabulary_size)
# standardize(["a cat on the sat, and i would like to tell with her."])

In [ ]:
# temp_captions = list(standardize(full_dataset_dict["2083434441_a93bc6306b.jpg"]))
# print(temp_captions)

['[START] an elderly woman is riding a bicycle in the city as a yellow taxi is about to pass by  [END]', '[START] an elderly woman rides a bicycle along a city street  [END]', '[START] an older woman with blond hair rides a bicycle down the street  [END]', '[START] a woman in a grey overcoat rides her bicycle along a street  [END]', '[START] older woman wearing glassses riding a bicycle with a shopping bag on the handle  yellow car is in the background  [END]']


In [37]:
preprocessing = preprocess_text()

In [20]:
for image in full_dataset_dict:
    full_dataset_dict[image] = preprocessing.standardize(full_dataset_dict[image])

In [21]:
full_dataset_dict["2083434441_a93bc6306b.jpg"]

['an elderly woman is riding a bicycle in the city as a yellow taxi is about to pass by  [END]',
 'an elderly woman rides a bicycle along a city street  [END]',
 'an older woman with blond hair rides a bicycle down the street  [END]',
 'a woman in a grey overcoat rides her bicycle along a street  [END]',
 'older woman wearing glassses riding a bicycle with a shopping bag on the handle  yellow car is in the background  [END]']

Make the vocabulary size

In [28]:
vocab_dict = preprocessing.make_vocabulary_dict(full_dataset_dict)

***Convert the dataset into the index***

In [41]:
full_dataset = {}
for image in full_dataset_dict:
    for caption in full_dataset_dict[image]:
        full_dataset[caption] = image

In [43]:
print(f'length of the full dataset each caption with image: {len(list(full_dataset.keys()))}')

length of the full dataset each caption with image: 40168


***Make the index to word ***

In [35]:
index_to_word = preprocessing.index_to_word(vocab_dict)

### ***Make the dataframe from the dictionary***

In [44]:
df = pd.DataFrame.from_dict(full_dataset, orient='index', columns=['image'])
df = df.reset_index().rename(columns={'index': 'caption'})
df

,caption,image
0,a child in a pink dress is climbing up a set o...,1000268201_693b08cb0e.jpg
1,a girl going into a wooden building [END],1000268201_693b08cb0e.jpg
2,a little girl climbing into a wooden playhouse...,1000268201_693b08cb0e.jpg
3,a little girl climbing the stairs to her playh...,1000268201_693b08cb0e.jpg
4,a little girl in a pink dress going into a woo...,1000268201_693b08cb0e.jpg
...,...,...
40163,woman writing on a pad in room with gold deco...,997338199_7343367d7f.jpg
40164,a man in a pink shirt climbs a rock face [END],997722733_0cb5439472.jpg
40165,a man is rock climbing high in the air [END],997722733_0cb5439472.jpg
40166,a person in a red shirt climbing up a rock fac...,997722733_0cb5439472.jpg


In [45]:
df['caption'] = df['caption'].apply(lambda x: x.replace("  ", " "))
df['caption'] = df['caption'].apply(lambda x: preprocessing.word_to_index(x))

In [46]:
df

,caption,image
0,"[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 1...",1000268201_693b08cb0e.jpg
1,"[1, 16, 17, 18, 1, 19, 20, 15]",1000268201_693b08cb0e.jpg
2,"[1, 21, 16, 7, 18, 1, 19, 22, 15]",1000268201_693b08cb0e.jpg
3,"[1, 21, 16, 7, 23, 11, 24, 25, 22, 15]",1000268201_693b08cb0e.jpg
4,"[1, 21, 16, 3, 1, 4, 5, 17, 18, 1, 19, 26, 15]",1000268201_693b08cb0e.jpg
...,...,...
40163,"[234, 1258, 38, 1, 4025, 3, 561, 35, 1202, 306...",997338199_7343367d7f.jpg
40164,"[1, 75, 3, 1, 4, 161, 112, 1, 164, 202, 15]",997722733_0cb5439472.jpg
40165,"[1, 75, 6, 164, 7, 488, 3, 23, 227, 15]",997722733_0cb5439472.jpg
40166,"[1, 187, 3, 1, 110, 161, 7, 8, 1, 164, 202, 54...",997722733_0cb5439472.jpg


In [ ]:
# # Convert dict keys and values to lists
# keys = list(full_dataset.keys())
# values = list(full_dataset.values())

# # Split keys and corresponding values together
# keys_train, keys_test, vals_train, vals_test = sklearn.model_selection.train_test_split(
#     keys, values,
#     test_size=0.2,
#     random_state=42,  # for reproducibility
#     shuffle=True
# )

# # Reconstruct train/test dicts
# train_data = dict(zip(keys_train, vals_train))
# test_data  = dict(zip(keys_test, vals_test))

In [ ]:
# print(f"Train data lengh: {len(list(train_data.keys()))}")
# print(f"Test data length: {len(list(test_data.keys()))}")

Train data lengh: 32134
Test data length: 8034


***Make the dataframe from the dictionary***

In [48]:
# df_train = pd.DataFrame.from_dict(train_data, orient='index', columns=['value'])
# df_train = df_train.reset_index().rename(columns={'index': 'key'})
# df_train

In [49]:
# df_test = pd.DataFrame.from_dict(test_data, orient='index', columns=['value'])
# df_test = df_test.reset_index().rename(columns={'index': 'key'})
# df_test

***Feature extractor that extract the feature from the image***

In [ ]:
def feature_extractor(img):
    resize_img = load_image(img)[tf.newaxis, :]
    img_feature = restnet(resize_img)
    return img_feature

## ***Transformer Embedding part of token***

In [50]:
len(list(vocab_dict.keys()))

8830

In [54]:
vocabulary_size = len(list(vocab_dict.keys()))
vocabulary_size

8830

In [ ]:
class SeqEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, max_length, depth):
    super().__init__()
    self.pos_embedding = tf.keras.layers.Embedding(input_dim=max_length, output_dim=depth)

    self.token_embedding = tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=depth,
        mask_zero=True)

    self.add = tf.keras.layers.Add()

    self.max_length = max_length

  def __call__(self, seq):
    # seq = convert_into_index(seq)
    print(f"Calling Sequence embedding: {seq}")

    for _ in range(0, self.max_length-len(seq)):
        seq.append(0)

    seq = np.array(seq, dtype=np.int32)
    print(f"Sequence for embedding: {seq}")
    seq = self.token_embedding(seq) # (batch, seq, depth)

    print(f"After token embedding: {seq.shape}\n")

    token_emb = seq

    x = tf.range(tf.shape(seq)[0])  # (seq)
    x = x[tf.newaxis, :]  # (1, seq)
    print(x.shape)
    x = self.pos_embedding(x)  # (1, seq, depth)

    print(f"After positional embedding: {x.shape}\n")

    return self.add([seq, token_emb[np.newaxis, :, :]])

In [ ]:
class CausalSelfAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    # Use Add instead of + so the keras mask propagates through.
    self.add = tf.keras.layers.Add()
    self.layernorm = tf.keras.layers.LayerNormalization()

  def __call__(self, x):
    attn = self.mha(query=x, value=x,
                    use_causal_mask=True)
    x = self.add([x, attn])
    return self.layernorm(x)

In [ ]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self,**kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.add = tf.keras.layers.Add()
    self.layernorm = tf.keras.layers.LayerNormalization()

  def __call__(self, x, y, **kwargs):
    attn, attention_scores = self.mha(
             query=x, value=y,
             return_attention_scores=True)

    self.last_attention_scores = attention_scores

    x = self.add([x, attn])
    return self.layernorm(x)

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, units, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
        tf.keras.layers.Dense(units=2*units, activation='relu'),
        tf.keras.layers.Dense(units=units),
        tf.keras.layers.Dropout(rate=dropout_rate),
    ])

    self.layernorm = tf.keras.layers.LayerNormalization()

  def __call__(self, x):
    x = x + self.seq(x)
    return self.layernorm(x)


In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, units, num_heads=1, dropout_rate=0.1):
    super().__init__()

    self.self_attention = CausalSelfAttention(num_heads=num_heads,
                                              key_dim=units,
                                              dropout=dropout_rate)
    self.cross_attention = CrossAttention(num_heads=num_heads,
                                          key_dim=units,
                                          dropout=dropout_rate)
    self.ff = FeedForward(units=units, dropout_rate=dropout_rate)


  def __call__(self, inputs, training=False):
    in_seq, out_seq = inputs

    # Text input
    out_seq = self.self_attention(out_seq)

    out_seq = self.cross_attention(out_seq, in_seq)

    self.last_attention_scores = self.cross_attention.last_attention_scores

    out_seq = self.ff(out_seq)

    return out_seq

In [ ]:
#@title
class TokenOutput(tf.keras.layers.Layer):
  def __init__(self, banned_tokens=('', '[UNK]', '[START]'), **kwargs):
    super().__init__()

    self.dense = tf.keras.layers.Dense(
        units=vocabulary_size, **kwargs)
    self.banned_tokens = banned_tokens

    self.bias = None

  def adapt(self, ds):
    counts = collections.Counter()

    for tokens in tqdm.tqdm(ds):
      counts.update(tokens)

    print(f"Counts in the Token Output: {counts}\n")

    counts_arr = np.zeros(shape=(vocabulary_size+1,))
    counts_arr[np.array(list(counts.keys()), dtype=np.int32)] = list(counts.values())

    counts_arr = counts_arr[:]
    for token in self.banned_tokens:
      counts_arr[vocab_dict[token]] = 0

    print(f"Counts array after removing banned tokens: {counts_arr}\n")
    print(f'Shape of the counts array: {counts_arr.shape}\n')

    total = counts_arr.sum()
    print(f"Total: {total}\n")
    p = counts_arr/total   ## get the empirical probability of each token. This is used to compute entropy and bias terms in probability space.
    print(f"Probability: {p}\n")
    p[counts_arr==0] = 1.0
    log_p = np.log(p)  # log(1) == 0

    print(f"log_p: {log_p}\n")

    entropy = -(log_p*p).sum()  ## Computes the marginal entropy. it describe how much uncertainty remains if you only knew the marginal token distribution.

    print()
    print(f"Uniform entropy: {np.log(vocabulary_size):0.2f}")
    print(f"Marginal entropy: {entropy:0.2f}")

    self.bias = log_p
    self.bias[counts_arr==0] = -1e9
    print(f"Bias: {self.bias}\n")

  def __call__(self, x):
    x = self.dense(x)
    # TODO(b/250038731): Fix this.
    # An Add layer doesn't work because of the different shapes.
    # This clears the mask, that's okay because it prevents keras from rescaling
    # the losses.
    return x + self.bias

In [ ]:
convert_into_index('the dog is playing with the stick on the beach')

[1, 24, 29, 7, 35, 36, 24, 318, 39, 24, 137, 16]

***Converting the caption into index***

In [ ]:
for index, caption in enumerate(df_train['key'], start=0):
    caption = caption.replace("  ", " ")
    caption = caption.replace("[END]", "")
    df_train.at[index, 'key'] = convert_into_index(caption)

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
for index, caption in enumerate(df_test['key'], start=0):
    caption = caption.replace("  ", " ")
    caption = caption.replace("[END]", "")
    df_test.at[index, 'key'] = convert_into_index(caption)

In [ ]:
df_train

,key,value
0,"[1, 2, 28, 29, 119, 39, 2, 675, 4, 24, 3928, 1...",3523920786_0eb63993fd.jpg
1,"[1, 24, 29, 7, 35, 36, 24, 318, 39, 24, 137, 16]",2479180530_7ebba2d8bf.jpg
2,"[1, 347, 439, 1756, 19, 2, 1114, 1754, 911, 36...",1362128028_8422d53dc4.jpg
3,"[1, 438, 198, 351, 4, 58, 11, 191, 16]",278002875_d011ae9dc5.jpg
4,"[1, 2, 42, 30, 41, 29, 7, 119, 9, 2, 120, 348,...",3361411074_83f27d2a1c.jpg
...,...,...
32129,"[1, 76, 39, 2, 681, 190, 45, 47, 48, 143, 4, 4...",2220185725_45d4fa68d9.jpg
32130,"[1, 2, 42, 29, 7, 175, 124, 2, 179, 16]",2558312618_13d362df66.jpg
32131,"[1, 2, 28, 29, 8629, 119, 387, 2, 163, 30, 95,...",537479916_c033897fac.jpg
32132,"[1, 2, 147, 77, 39, 2, 1144, 368, 78, 16]",1204996216_71d7519d9a.jpg


In [ ]:
for key, value in vocab_dict.items():
    print(f"{key}: {value}")
    if value == 2:
      break

[START]: 1
a: 2


In [ ]:
output_layer = TokenOutput(banned_tokens=['[START]'])
# This might run a little faster if the dataset didn't also have to load the image data.
output_layer.adapt(df_train['key'])

100%|██████████| 32134/32134 [00:00<00:00, 715570.05it/s]

Counts in the Token Output: Counter({2: 50217, 16: 32134, 4: 15120, 24: 14589, 39: 8561, 7: 7493, 30: 7105, 29: 6419, 36: 6211, 76: 5759, 11: 5377, 47: 4428, 41: 3167, 28: 3080, 147: 2916, 32: 2777, 235: 2695, 17: 2693, 25: 2523, 102: 2444, 45: 2354, 198: 2305, 142: 2155, 111: 2147, 74: 2111, 42: 2036, 13: 1965, 80: 1907, 163: 1826, 48: 1649, 119: 1623, 79: 1607, 124: 1598, 35: 1576, 162: 1483, 387: 1449, 143: 1436, 135: 1426, 22: 1424, 66: 1244, 3: 1243, 188: 1231, 175: 1183, 178: 1171, 167: 1127, 58: 1123, 347: 1113, 63: 1080, 92: 1053, 65: 1032, 9: 1004, 179: 998, 81: 989, 64: 981, 128: 957, 223: 952, 197: 950, 145: 950, 26: 940, 156: 926, 439: 912, 275: 881, 228: 844, 137: 835, 19: 823, 130: 807, 224: 787, 231: 783, 46: 770, 186: 766, 193: 750, 211: 745, 132: 740, 139: 714, 647: 712, 129: 708, 436: 687, 295: 683, 549: 667, 75: 634, 375: 619, 5: 618, 51: 617, 95: 612, 38: 607, 165: 601, 141: 599, 87: 596, 277: 590, 1232: 574, 257: 560, 491: 553, 290: 542, 357: 540, 907: 528, 458: 52

In [ ]:
df_test

,key,value
0,"[1, 2, 433, 36, 47, 3432, 57, 3433, 39, 24, 23...",215214751_e913b6ff09.jpg
1,"[1, 2, 28, 30, 41, 29, 231, 25, 230, 24, 523, 16]",1067180831_a59dc64344.jpg
2,"[1, 2, 29, 129, 39, 24, 137, 36, 2, 111, 29, 2...",2364096157_eb7970a69a.jpg
3,"[1, 2, 1232, 39, 24, 41, 30, 163, 1115, 2508, ...",1597319381_1e80d9e39c.jpg
4,"[1, 47, 28, 30, 42, 48, 32, 119, 124, 2, 179, 16]",2439384468_58934deab6.jpg
...,...,...
8029,"[1, 2, 197, 11, 439, 35, 36, 4670, 16]",494907021_321e82877a.jpg
8030,"[1, 2, 65, 29, 1920, 39, 2, 237, 29, 233, 25, ...",2414397449_2ac3b78e0d.jpg
8031,"[1, 2, 22, 17, 102, 2, 163, 6, 139, 4, 58, 11,...",3091594712_2166604334.jpg
8032,"[1, 2, 1049, 2148, 417, 3958, 16]",3175712926_5a470d0886.jpg


***Remove the ['START'] Token from the key in train and test dataset.***

In [ ]:
df_train['key'] = [a[1:] for a in df_train['key']]
df_test['key'] = [a[1:] for a in df_test['key']]

In [ ]:
class Captioner(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, tokenizer, feature_extractor, output_layer, index_to_word, num_layers=1,
               units=256, max_length=50, num_heads=1, dropout_rate=0.1):
    super().__init__()
    self.feature_extractor = feature_extractor
    # self.word_to_index = tf.keras.layers.StringLookup(
    #     mask_token="",
    #     vocabulary=tokenizer.get_vocabulary())
    self.index_to_word = index_to_word

    self.seq_embedding = SeqEmbedding(
        vocab_size=vocabulary_size,
        depth=units,
        max_length=max_length)

    self.decoder_layers = [
        DecoderLayer(units, num_heads=num_heads, dropout_rate=dropout_rate)
        for n in range(num_layers)]

    self.output_layer = output_layer

In [ ]:
  @Captioner.add_method
  def __call__(self, image, txt):
    if image.shape[-1] == 3:
      # Apply the feature-extractor, if you get an RGB image.
      image = self.feature_extractor(image)

    # Flatten the feature map
    image = tf.reshape(image, [image.shape[0], -1])

    print(image)
    txt = self.seq_embedding(txt)

    # Look at the image
    for dec_layer in self.decoder_layers:
      txt = dec_layer(inputs=(image, txt))

    txt = self.output_layer(txt)

    return txt

In [ ]:
model = Captioner(standardize, feature_extractor=feature_extractor, output_layer=output_layer, index_to_word=index_to_word,
                  units=256, dropout_rate=0.5, num_layers=2, num_heads=2)

In [ ]:
@Captioner.add_method
def simple_gen(self, image, temperature=1):
  initial = [vocab_dict['[START]']] # (batch, sequence)
  img_features = self.feature_extractor(image)

  tokens = initial # (batch, sequence)
  print(tokens)
  for n in range(50):
    preds = self(img_features, tokens).numpy()  # (batch, sequence, vocab)
    preds = preds[:,-1, :]  #(batch, vocab)
    if temperature==0:
        next = tf.argmax(preds, axis=-1)[:, tf.newaxis]  # (batch, 1)
    else:
        next = tf.random.categorical(preds/temperature, num_samples=1)  # (batch, 1)
    tokens = tf.concat([tokens, next], axis=1) # (batch, sequence)

    if next[0] == vocab_dict['[END]']:
      break
  words = index_to_word(tokens[0, 1:-1])
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  return result.numpy().decode()

In [ ]:
vocabulary_size

8831

In [ ]:
for t in (0.0, 0.5, 1.0):
  result = model.simple_gen("215214751_e913b6ff09.jpg", temperature=t)
  print(result)

[1]
tf.Tensor([[0.       0.       9.063452 ... 0.       0.       0.      ]], shape=(1, 100352), dtype=float32)
Calling Sequence embedding: [1]
Sequence for embedding: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
After token embedding: (50, 256)

(1, 50)
After positional embedding: (1, 50, 256)



ValueError: Cannot merge tensors with different batch sizes. Received tensors with shapes [(50, 256), (1, 50, 256)]

In [61]:
import math
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        # Create a long enough P matrix once
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len,1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))  # (d_model/2,)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape (batch_size, seq_len, d_model)
        Returns:
            x + positional encoding, same shape
        """
        seq_len = x.size(1)
        # only take the first seq_len positions
        return x + self.pe[:, :seq_len]

class DecoderBlock(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, nhead: int):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, d_model)
        self.pos_enc     = PositionalEncoding(d_model)
        # this single layer can serve as demonstration; in practice you’d stack several of these
        self.self_attn   = nn.MultiheadAttention(d_model, nhead)
        self.ffn         = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.ReLU(),
            nn.Linear(4*d_model, d_model)
        )
        self.layernorm1  = nn.LayerNorm(d_model)
        self.layernorm2  = nn.LayerNorm(d_model)

    def forward(self, tgt: torch.Tensor) -> torch.Tensor:
        """
        Args:
            tgt: target token IDs, shape (batch_size, tgt_len)
        Returns:
            output embeddings, shape (batch_size, tgt_len, d_model)
        """
        bsz, tgt_len = tgt.size()

        # 1) Token embedding
        tok_emb = self.token_embed(tgt)                     # (bsz, tgt_len, d_model)

        # 2) Positional embedding
        x = self.pos_enc(tok_emb)                           # (bsz, tgt_len, d_model)

        return x

# Example usage
if __name__ == "__main__":
    B, L, V, D, H = 2, 10, 1000, 512, 8
    decoder = DecoderBlock(vocab_size=V, d_model=D, nhead=H)
    test_tensor = convert_output_to_tensor(vocab_dict=vocab_dict, vocabulary_size=vocabulary_size)
    tgt_ids = torch.tensor([[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 12, 13, 14, 15], [1, 16, 17, 18, 1, 19, 20, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.long)
    out = decoder(tgt_ids)                                # (B, L, D)
    print(out.shape)  # -> torch.Size([2, 10, 512])


torch.Size([2, 18, 512])


In [58]:
df

,caption,image
0,"[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 1...",1000268201_693b08cb0e.jpg
1,"[1, 16, 17, 18, 1, 19, 20, 15]",1000268201_693b08cb0e.jpg
2,"[1, 21, 16, 7, 18, 1, 19, 22, 15]",1000268201_693b08cb0e.jpg
3,"[1, 21, 16, 7, 23, 11, 24, 25, 22, 15]",1000268201_693b08cb0e.jpg
4,"[1, 21, 16, 3, 1, 4, 5, 17, 18, 1, 19, 26, 15]",1000268201_693b08cb0e.jpg
...,...,...
40163,"[234, 1258, 38, 1, 4025, 3, 561, 35, 1202, 306...",997338199_7343367d7f.jpg
40164,"[1, 75, 3, 1, 4, 161, 112, 1, 164, 202, 15]",997722733_0cb5439472.jpg
40165,"[1, 75, 6, 164, 7, 488, 3, 23, 227, 15]",997722733_0cb5439472.jpg
40166,"[1, 187, 3, 1, 110, 161, 7, 8, 1, 164, 202, 54...",997722733_0cb5439472.jpg


In [57]:
df['caption'][0]

[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 12, 13, 14, 15]